In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('sgan')):
    os.mkdir('saved_models_{}'.format('sgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('sgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')


if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=50, batch_size=64, sample_interval=50)



Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.677287, acc: 32.03%, op_acc: 6.25%] [G loss: 0.657322]
epoch:0 step:2[D loss: 0.604413, acc: 46.88%, op_acc: 8.59%] [G loss: 0.685917]
epoch:0 step:3[D loss: 0.582296, acc: 56.25%, op_acc: 5.47%] [G loss: 0.801060]
epoch:0 step:4[D loss: 0.566151, acc: 55.47%, op_acc: 11.72%] [G loss: 0.823084]
epoch:0 step:5[D loss: 0.545274, acc: 63.28%, op_acc: 13.28%] [G loss: 0.849048]
epoch:0 step:6[D loss: 0.578087, acc: 51.56%, op_acc: 10.94%] [G loss: 0.795171]
epoch:0 step:7[D loss: 0.546784, acc: 56.25%, op_acc: 12.50%] [G loss: 0.825469]
epoch:0 step:8[D loss: 0.548622, acc: 65.62%, op_acc: 7.03%] [G loss: 0.844846]
epoch:0 step:9[D loss: 0.537477, acc: 58.59%, op_acc: 10.94%] [G loss: 1.012536]
epoch:0 step:10[D loss: 0.502473, acc: 67.97%, op_acc: 14.06%] [G loss: 0.974035]
epoch:0 step:11[D loss: 0.522098, acc: 66.41%, op_acc: 14.84%] [G loss: 0.908690]
epoch:0 step:12[D loss: 0.515360, acc: 65.62%, op_acc: 12.50%] [G loss: 0.949969]
epoch:0 step:13[D loss: 0.507

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

##############
[0.85714192 0.85295047 0.82035626 0.82188592 0.79645566 0.82681171
 0.9050024  0.80289919 0.80533346 0.82630024]
##########
epoch:49 step:38651[D loss: 0.376485, acc: 71.09%, op_acc: 46.88%] [G loss: 0.895994]
epoch:49 step:38652[D loss: 0.385305, acc: 69.53%, op_acc: 46.09%] [G loss: 0.844826]
epoch:49 step:38653[D loss: 0.416280, acc: 59.38%, op_acc: 46.09%] [G loss: 1.083769]
epoch:49 step:38654[D loss: 0.330395, acc: 67.97%, op_acc: 53.91%] [G loss: 1.066307]
epoch:49 step:38655[D loss: 0.358347, acc: 72.66%, op_acc: 57.81%] [G loss: 1.131190]
epoch:49 step:38656[D loss: 0.408045, acc: 66.41%, op_acc: 44.53%] [G loss: 1.105981]
epoch:49 step:38657[D loss: 0.451900, acc: 60.16%, op_acc: 35.16%] [G loss: 0.741598]
epoch:49 step:38658[D loss: 0.365978, acc: 69.53%, op_acc: 48.44%] [G loss: 0.861117]
epoch:49 step:38659[D loss: 0.376378, acc: 70.31%, op_acc: 46.88%] [G loss: 1.126327]
epoch:49 step:38660[D loss: 0.394652, acc: 64.06%, op_acc: 46.09%] [G loss: 1.060530]
e

epoch:49 step:38748[D loss: 0.335198, acc: 75.78%, op_acc: 46.09%] [G loss: 0.779535]
epoch:49 step:38749[D loss: 0.390432, acc: 67.19%, op_acc: 39.06%] [G loss: 1.227673]
epoch:49 step:38750[D loss: 0.406001, acc: 66.41%, op_acc: 39.06%] [G loss: 0.703087]
##############
[0.85636073 0.85965867 0.8309406  0.81563885 0.79995948 0.80999321
 0.91768792 0.80934821 0.78399724 0.81417142]
##########
epoch:49 step:38751[D loss: 0.404493, acc: 64.06%, op_acc: 42.97%] [G loss: 0.892449]
epoch:49 step:38752[D loss: 0.339991, acc: 74.22%, op_acc: 47.66%] [G loss: 0.789308]
epoch:49 step:38753[D loss: 0.342809, acc: 73.44%, op_acc: 58.59%] [G loss: 0.710441]
epoch:49 step:38754[D loss: 0.395163, acc: 67.19%, op_acc: 38.28%] [G loss: 0.803332]
epoch:49 step:38755[D loss: 0.357455, acc: 65.62%, op_acc: 53.91%] [G loss: 0.917536]
epoch:49 step:38756[D loss: 0.311154, acc: 85.16%, op_acc: 50.78%] [G loss: 1.391027]
epoch:49 step:38757[D loss: 0.410373, acc: 63.28%, op_acc: 42.19%] [G loss: 0.897610]
e

epoch:49 step:38844[D loss: 0.392837, acc: 65.62%, op_acc: 40.62%] [G loss: 0.979556]
epoch:49 step:38845[D loss: 0.343201, acc: 71.09%, op_acc: 46.88%] [G loss: 1.118750]
epoch:49 step:38846[D loss: 0.379227, acc: 71.09%, op_acc: 46.88%] [G loss: 1.164456]
epoch:49 step:38847[D loss: 0.409675, acc: 64.84%, op_acc: 41.41%] [G loss: 1.074273]
epoch:49 step:38848[D loss: 0.328147, acc: 73.44%, op_acc: 53.91%] [G loss: 0.683559]
epoch:49 step:38849[D loss: 0.381457, acc: 71.09%, op_acc: 45.31%] [G loss: 0.960400]
epoch:49 step:38850[D loss: 0.411690, acc: 60.94%, op_acc: 42.97%] [G loss: 1.205289]
##############
[0.84362048 0.87087324 0.82303165 0.81254686 0.79882421 0.83179998
 0.8874002  0.80993673 0.83233616 0.82144093]
##########
epoch:49 step:38851[D loss: 0.403742, acc: 62.50%, op_acc: 46.88%] [G loss: 1.041588]
epoch:49 step:38852[D loss: 0.382025, acc: 62.50%, op_acc: 45.31%] [G loss: 1.051538]
epoch:49 step:38853[D loss: 0.400924, acc: 65.62%, op_acc: 40.62%] [G loss: 0.956461]
e

epoch:49 step:38941[D loss: 0.418302, acc: 57.81%, op_acc: 46.88%] [G loss: 0.854357]
epoch:49 step:38942[D loss: 0.374712, acc: 68.75%, op_acc: 51.56%] [G loss: 0.849889]
epoch:49 step:38943[D loss: 0.362693, acc: 71.88%, op_acc: 46.09%] [G loss: 1.120819]
epoch:49 step:38944[D loss: 0.311995, acc: 75.78%, op_acc: 52.34%] [G loss: 0.774049]
epoch:49 step:38945[D loss: 0.393613, acc: 65.62%, op_acc: 50.78%] [G loss: 0.863574]
epoch:49 step:38946[D loss: 0.420228, acc: 61.72%, op_acc: 34.38%] [G loss: 1.093057]
epoch:49 step:38947[D loss: 0.319876, acc: 75.00%, op_acc: 50.78%] [G loss: 0.873377]
epoch:49 step:38948[D loss: 0.332269, acc: 72.66%, op_acc: 47.66%] [G loss: 0.835384]
epoch:49 step:38949[D loss: 0.336729, acc: 80.47%, op_acc: 54.69%] [G loss: 1.116465]
epoch:49 step:38950[D loss: 0.347684, acc: 78.12%, op_acc: 40.62%] [G loss: 1.199158]
##############
[0.86563389 0.86477758 0.81089694 0.81154656 0.79810687 0.83509973
 0.87542535 0.80086118 0.78616936 0.81934221]
##########
e

epoch:49 step:39037[D loss: 0.397260, acc: 64.84%, op_acc: 46.09%] [G loss: 1.098385]
epoch:49 step:39038[D loss: 0.370103, acc: 67.19%, op_acc: 50.00%] [G loss: 1.114055]
epoch:49 step:39039[D loss: 0.377742, acc: 62.50%, op_acc: 48.44%] [G loss: 1.127146]
epoch:49 step:39040[D loss: 0.401035, acc: 67.19%, op_acc: 44.53%] [G loss: 1.111039]
epoch:49 step:39041[D loss: 0.448767, acc: 60.16%, op_acc: 35.16%] [G loss: 1.121867]
epoch:49 step:39042[D loss: 0.405152, acc: 66.41%, op_acc: 42.19%] [G loss: 1.049948]
epoch:49 step:39043[D loss: 0.378202, acc: 65.62%, op_acc: 39.84%] [G loss: 0.962531]
epoch:49 step:39044[D loss: 0.421311, acc: 64.06%, op_acc: 44.53%] [G loss: 0.801819]
epoch:49 step:39045[D loss: 0.377507, acc: 67.19%, op_acc: 50.00%] [G loss: 0.925910]
epoch:49 step:39046[D loss: 0.381719, acc: 73.44%, op_acc: 45.31%] [G loss: 0.879246]
epoch:49 step:39047[D loss: 0.327967, acc: 77.34%, op_acc: 50.78%] [G loss: 0.841625]
epoch:49 step:39048[D loss: 0.359176, acc: 68.75%, op_